In [2]:
import ollama

modelfile = {
    "temperature": 0.7,
    "template": "{{ if .System }}<|start_header_id|>system<|end_header_id|>\n\n{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>\n\n{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>Alice<|end_header_id|>\n\n{{ .Response }}<|eot_id|>",
    }

response = ollama.chat(
                    model="llama3.2:1b",
                    messages=[
                        {
                            "role": "system", 
                            "content": "\n[Action Options]\n1) PICK <obj> PATH <path>: only PICK if your gripper is empty;\n2) PLACE <obj> bin PATH <path>: only if you have already PICKed the object, you can PLACE it into an empty bin slot, do NOT PLACE if another object is already in a slot!\n\nEach <path> must contain exactly four <coord>s that smoothly interpolate between start and goal, coordinates must be evenly distanced from each other.\nThe robot PATHs must efficiently reach target while avoiding collision avoid collision (e.g. move above the objects' heights).\nThe PATHs must do top-down pick or place: \n- move directly atop an object by height 0.2 before PICK: e.g. Alice's gripper is at (0, 0, 0.3), banana is at (-0.25, 0.39, 0.29): NAME Alice ACTION PICK banana PATH [(0, 0.1, 0.3),(0, 0.2, 0.49),(-0.1, 0.25, 0.49),(-0.25, 0.39, 0.49)]\n- lift an object vertically up before moving it to PLACE: e.g. Bob's gripper is at (0.9, 0, 0.2), bin_front_left is at (0.35, 0.35, 0.43): NAME Bob ACTION PLACE apple bin_front_left PATH [(0.9,0.0,0.5), (0.5, 0, 0.5), (0.2, 0.1, 0.5),(0.35, 0.35, 0.5)]\n\n[Action Output Instruction]\nFirst output 'EXECUTE\n', then give exactly one ACTION per robot, each on a new line.\nExample: 'EXECUTE\nNAME Alice ACTION PICK apple PATH <path>\nNAME Bob ACTION PLACE banana bin_back_middle PATH <path>\n'\n\n[Path Plan Instruction]\nEach <coord> is a tuple (x,y,z) for gripper location, follow these steps to plan:\n1) Decide target location (e.g. an object you want to pick), and your current gripper location.\n2) Plan a list of <coord> that move smoothly from current gripper to the target location.\n3) The <coord>s must be evenly spaced between start and target.\n4) Each <coord> must not collide with other robots, and must stay away from table and objects.  \n[How to Incoporate [Enviornment Feedback] to improve plan]\n    If IK fails, propose more feasible step for the gripper to reach. \n    If detected collision, move robot so the gripper and the inhand object stay away from the collided objects. \n    If collision is detected at a Goal Step, choose a different action.\n    To make a path more evenly spaced, make distance between pair-wise steps similar.\n        e.g. given path [(0.1, 0.2, 0.3), (0.2, 0.2. 0.3), (0.3, 0.4. 0.7)], the distance between steps (0.1, 0.2, 0.3)-(0.2, 0.2. 0.3) is too low, and between (0.2, 0.2. 0.3)-(0.3, 0.4. 0.7) is too high. You can change the path to [(0.1, 0.2, 0.3), (0.15, 0.3. 0.5), (0.3, 0.4. 0.7)] \n    If a plan failed to execute, re-plan to choose more feasible steps in each PATH, or choose different actions.\n\n\n\nYou are Alice, you and robot Bob each stands at a different side of the table, and together you must put all the grocery items into a bin.\nLocations of slots in the bin:\nbin_front_left: (0.25, 0.41, 0.36)\nbin_front_right: (0.65, 0.41, 0.36)\nbin_front_middle: (0.45, 0.41, 0.36)\nbin_back_left: (0.25, 0.58, 0.36)\nbin_back_right: (0.65, 0.58, 0.36)\nbin_back_middle: (0.45, 0.58, 0.36)\nAt current round:\nYou see the following objects:\napple: (-0.73, 0.40, 0.30), on table\nbanana: (-0.25, 0.39, 0.27), on table\nmilk: (-0.01, 0.57, 0.47), on table\nsoda_can: (-0.72, 0.62, 0.33), on table\nbread: (-0.22, 0.60, 0.31), on table\ncereal: (-0.04, 0.38, 0.44), on table\nYour gripper: (0.29, 0.06, 0.51), holding nothing\nYour gripper must move higher than these objects and higher than table height 0.25, but move lower than 0.8.\nNever forget you are Alice!\nThink step-by-step about the task and Bob's response. Carefully check and correct Bob if they made a mistake. \nDiscuss with Bob to come up with the best plan and smooth, collision-free paths. \nImprove your paths if given [Environment Feedback], choose a different object or target slot if needed.\n\nWhen you respond, tell Bob about your status. Respond very concisely but informatively, and do not repeat what others have said.\nPropose exactly one action for yourself at the **current** round, select from [Action Options].\nEnd your response by either: 1) output PROCEED, if the plans require further discussion; 2) If everyone has made proposals and got approved, output the final plan, must strictly follow [Action Output Instruction] and [Path Plan Instruction].\n\n\n"
                        },
                        {
                            "role": "user",
                            "content": "You are Alice, your response is:"
                        },
                        {
                            "role": "Alice",
                            "content": ""
                        }
                    ],
                    options=modelfile,
                    stream=False
                )
print(response)
print("Test")

{'model': 'llama3.2:1b', 'created_at': '2024-10-13T20:39:38.098215257Z', 'message': {'role': 'assistant', 'content': "<|start_header_id|>assistant<|end_header_id|>\n\nI am at (0.29, 0.06, 0.51) with my gripper holding nothing.\n\nFirst, I need to decide which object I want to pick up. Since there are several objects on the table that could be picked, let's consider them one by one.\n\nThe apple is located at (-0.73, 0.40, 0.30), and it's currently below my gripper height of 0.25. The banana is also at (-0.25, 0.39, 0.27), which is above my current height. However, the milk is closer to me (my gripper height) than the apple or banana but too far from table height 0.25.\n\nThe soda can is located at (-0.72, 0.62, 0.33), and it's also below my gripper height of 0.25. The bread is at (-0.22, 0.60, 0.31), which is above my current height. Similarly, the cereal is close to me (my gripper height) but too far from table height 0.25.\n\nBased on this analysis, I decide to pick up the apple. I w

In [3]:
response["message"]["content"]

'EXECUTE\nNAME Alice ACTION PICK apple PATH [(0.29, 0.06, 0.8), (0.2, -0.05, 0.7), (-0.1, -0.05, 0.65), (-0.73, 0.40, 0.5)]\nNAME Alice ACTION PLACE apple bin_front_left PATH [(0.25, 0.41, 0.4), (0.3, 0.42, 0.45), (0.2, 0.39, 0.46), (0.25, 0.41, 0.36)]'